# Light Training: Few-Shot Learning

**Thiết lập tập theo tập (episodic).** Với bài toán \\(N\\)-way \\(K\\)-shot, trong mỗi tập:

- **Tập hỗ trợ (support)**: \\( \mathcal{S}=\{(x_{c,i}^{\mathrm{s}},\, y_{c}^{\mathrm{s}}=c)\}_{c=1..N,\, i=1..K} \\)
- **Tập truy vấn (query)**: \\( \mathcal{Q}=\{(x_{c,j}^{\mathrm{q}},\, y_{c}^{\mathrm{q}}=c)\}_{c=1..N,\, j=1..Q} \\)

Gọi **bộ mã hoá** (encoder) gọn nhẹ là \\( f_\theta:\mathbb{R}^{3\times32\times32}\to\mathbb{R}^d \\),
\\( z=f_\theta(x)\in\mathbb{R}^d \\).
Chuẩn hoá cosine: \\( \hat{z}=z/\|z\|_2 \\).

**Độ đo/điểm tương tự** giữa hai đặc trưng \\(u,v\\):

- **Khoảng cách Euclid**: \\( d_{\mathrm{E}}(u,v)=\|u-v\|_2 \\)
- **Cosine**: \\( s_{\cos}(u,v)=\frac{u^\top v}{\|u\|_2\|v\|_2}=\hat{u}^\top \hat{v} \\)

**Xác suất softmax có nhiệt độ** \\( \tau>0 \\) với các “logits” \\( \{\ell_c\} \\):
\\[
\mathrm{softmax}_\tau(\ell)_c=\frac{\exp(\ell_c/\tau)}{\sum_{c'}\exp(\ell_{c'}/\tau)}.
\\]

**Độ chính xác Top-k** cho tập \\( \mathcal{A} \\):
\\[
\mathrm{Top}@k(\mathcal{A})=\frac{1}{|\mathcal{A}|}\sum_{(x,y)\in\mathcal{A}} \mathbf{1}\{y \in \text{k lớp có xác suất cao nhất cho } x\}.
\\]


## (A) Prototypical Networks

**Prototype mỗi lớp** \\( c \\):
\\[
\mu_c \;=\; \frac{1}{K}\sum_{i=1}^K f_\theta\!\left(x_{c,i}^{\mathrm{s}}\right) \in \mathbb{R}^d.
\\]

**Logits & phân phối dự đoán cho truy vấn** \\( x^{\mathrm{q}} \\) (hai biến thể):

- **Proto-Euclidean** (dùng khoảng cách âm):
\\[
\ell_c(x^{\mathrm{q}}) \;=\; -\, d_{\mathrm{E}}\!\left(f_\theta(x^{\mathrm{q}}),\, \mu_c\right),
\qquad
p_\theta\!\left(y=c\mid x^{\mathrm{q}}\right)\;=\;\mathrm{softmax}_\tau\!\left(\{\ell_c\}\right).
\\]

- **Proto-Cosine** (cosine + hệ số tỉ lệ \\( \alpha \\)):
\\[
\ell_c(x^{\mathrm{q}}) \;=\; \alpha \cdot s_{\cos}\!\left(f_\theta(x^{\mathrm{q}}),\, \mu_c\right),
\qquad
p_\theta\!\left(y=c\mid x^{\mathrm{q}}\right)\;=\;\mathrm{softmax}_\tau\!\left(\{\ell_c\}\right).
\\]

**Hàm mất mát theo tập (episodic cross-entropy)**:
\\[
\mathcal{L}_{\text{proto}}(\theta)
\;=\; - \frac{1}{NQ} \sum_{c=1}^N \sum_{j=1}^Q 
\log p_\theta\!\left(y=c \mid x_{c,j}^{\mathrm{q}}\right).
\\]

**Đánh giá:** Top-1/Top-5 trên \\( \mathcal{Q} \\) (val/test).  
**Checkpoint:** `fsl_prototypical_networks_eucl.pth`, `fsl_prototypical_networks_cos.pth`.


## (B) Siamese Networks

Cho đặc trưng \\( z=f_\theta(x) \\), khoảng cách \\( d(u,v) = \|u-v\|_2 \\).

### (B1) Contrastive Loss (học trên cặp)
Với cặp \\( (x_1,x_2) \\) và nhãn cặp \\( y\in\{0,1\} \\) (\\(1\\): cùng lớp, \\(0\\): khác lớp), margin \\( m>0 \\):
\\[
\mathcal{L}_{\text{contrast}}(\theta)
=\; y\; d\!\big(z_1,z_2\big)^2 \;+\; (1-y)\; \big[\,\max(0, m - d(z_1,z_2))\,\big]^2.
\\]

### (B2) Triplet Loss (học trên bộ 3)
Với bộ \\( (x^{\mathrm{a}}, x^{\mathrm{p}}, x^{\mathrm{n}}) \\) (anchor, positive cùng lớp; negative khác lớp):
\\[
\mathcal{L}_{\text{triplet}}(\theta)
=\; \max\!\big(0, \; d(z^{\mathrm{a}},z^{\mathrm{p}}) \;-\; d(z^{\mathrm{a}},z^{\mathrm{n}}) \;+\; m \big).
\\]

**Suy luận (inference):** phân loại bằng k-NN/cosine-NN trong không gian \\( f_\theta \\) hoặc huấn luyện **linear probe** nhỏ trên \\( f_\theta(x) \\).

**Đánh giá:** Top-1/Top-5 theo k-NN (val/test) hoặc theo probe.  
**Checkpoint:** `fsl_siamese_networks_contrastive.pth`, `fsl_siamese_networks_triplet.pth`.


## (C) Matching Networks (Cosine Attention)

Với **support** \\( \mathcal{S}=\{(x_i^{\mathrm{s}}, y_i^{\mathrm{s}})\}_{i=1}^{NK} \\),
đặc trưng \\( s_i=f_\theta(x_i^{\mathrm{s}}) \\) và **query** \\( q=f_\theta(x^{\mathrm{q}}) \\).

**Attention theo cosine** cho từng phần tử support:
\\[
a_i(q) \;=\; \frac{\exp\!\left(\alpha\, s_{\cos}\!\left(q,\, s_i\right)\right)}
{\sum\limits_{j=1}^{NK}\exp\!\left(\alpha\, s_{\cos}\!\left(q,\, s_j\right)\right)}.
\\]

**Phân phối dự đoán của query** (gộp theo nhãn support):
\\[
p_\theta\!\left(y=c\mid x^{\mathrm{q}}\right)
\;=\; \sum_{i=1}^{NK} a_i(q)\; \mathbf{1}[\,y_i^{\mathrm{s}}=c\,].
\\]

**Hàm mất mát (negative log-likelihood) theo tập**:
\\[
\mathcal{L}_{\text{match}}(\theta)
\;=\; - \frac{1}{NQ} \sum_{c=1}^N \sum_{j=1}^Q 
\log p_\theta\!\left(y=c \mid x_{c,j}^{\mathrm{q}}\right).
\\]

**Ghi chú:** Biến thể ở đây **không** dùng FCE/LSTM để giữ mô hình **nhẹ**; chỉ dùng attention cosine.  
**Checkpoint:** `fsl_matching_networks_cosine.pth`.
